In [1]:
from sentence_transformers import SentenceTransformer,util
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import json
import torch

data = pd.read_csv("data/climate_fever_evidence_embedding.csv",header=None)
embds = []

for embd in data[1]:
    embds.append(json.loads(embd))

embds = torch.Tensor(embds)

model = SentenceTransformer('sentence-transformers/all-roberta-large-v1')

def topkRelatedSentence(k, inputEmb, dataEmb):
    similarityScore = util.cos_sim(inputEmb, dataEmb)
    return torch.topk(similarityScore, k)[1].reshape(-1)

c:\Users\enzol\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\enzol\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\enzol\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
input = "Brooks added that Antarctic ice is growing."

print("Claim :\n")
print(input)

indexes = topkRelatedSentence(5, model.encode(input), embds)

topEvidences = data[0].iloc[indexes].tolist()

print("\nTop 5 related sentences with cosim from embeddings :\n")

for evidence in topEvidences:
    print(evidence)

Claim :

Brooks added that Antarctic ice is growing.

Top 5 related sentences with cosim from embeddings :

In contrast to the melting of the Arctic sea ice, sea ice around Antarctica has been expanding as of 2013[update].
"A 40-y record reveals gradual Antarctic sea ice increases followed by decreases at rates far exceeding the rates seen in the Arctic".
"Study concludes Antarctica is gaining ice, rather than losing it".
"Mass gains of the Antarctic ice sheet exceed losses".
Arctic temperatures have increased and are predicted to continue to increase during this century at over twice the rate of the rest of the world.
